# Perform Necessary Imports

In [20]:
import pandas as pd
import xgboost
import re
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score
import pickle

In [2]:
# !pip install xgboost numpy scikit-learn

# Read filtered Dataset

In [5]:
totalData = pd.read_csv('nela10.csv')

# How it looks like?

In [6]:
totalData


,id,date,source,title,content,author,url,published,published_utc,collection_utc,Reliability
0,abcnews--2019-01-30--Who is Stacey Abrams the ...,2019-01-30,abcnews,"Who is Stacey Abrams, the Democrat who will re...","Last November, Stacey Abrams, a Democrat who r...",Cheyenne Haslett,https://abcnews.go.com/Politics/stacey-abrams-...,2019-01-30 19:10:47+00:00,1548893447,1567550233,0
1,abcnews--2019-01-30--Ban government shutdowns ...,2019-01-30,abcnews,Ban government shutdowns? Some Republicans and...,As lawmakers scramble to negotiate a border se...,"Benjamin Siegel,\nTrish Turner\n",https://abcnews.go.com/Politics/ban-government...,2019-01-30 17:29:04+00:00,1548887344,1567550233,0
2,abcnews--2019-01-30--Democrats renew push for ...,2019-01-30,abcnews,Democrats renew push for equal pay for equal work,Ten years after President Barack Obama signed ...,John Parkinson,https://abcnews.go.com/Politics/dems-renew-pus...,2019-01-30 21:35:45+00:00,1548902145,1567550233,0
3,abcnews--2019-01-30--Trump attacks intel chief...,2019-01-30,abcnews,Trump attacks intel chiefs after they contradi...,"President Donald Trump on Wednesday declared ""...",Meridith Mcgraw,https://abcnews.go.com/Politics/trump-attacks-...,2019-01-30 17:03:40+00:00,1548885820,1567550233,0
4,abcnews--2019-01-30--Border security battle be...,2019-01-30,abcnews,Border security battle begins as Congress grap...,Republicans and Democrats on the exclusive com...,"John Parkinson,\nBenjamin Siegel\n",https://abcnews.go.com/Politics/border-securit...,2019-01-30 15:09:47+00:00,1548878987,1567550233,0
...,...,...,...,...,...,...,...,...,...,...,...
57152,zerohedge--2019-11-01--Hong Kong Officer Faces...,2019-11-01,zerohedge,Hong Kong Officer Faces Death Threats After Fi...,It's unclear exactly when it happened (the BBG...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 22:45:00 +0000",1572662700,1572648625,2
57153,"zerohedge--2019-11-01--Enough ""Quid Pro Quo"" G...",2019-11-01,zerohedge,"Enough ""Quid Pro Quo"" Gaslighting!",Horse trading is the oxygen of politics; it is...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 22:25:00 +0000",1572661500,1572648625,2
57154,"zerohedge--2019-11-01--""Born For This? I Don't...",2019-11-01,zerohedge,"""Born For This? I Don't Think So"" - Trump Mock...",Having plunged from over 10% to just 1% in the...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 21:56:35 +0000",1572659795,1572648626,2
57155,zerohedge--2019-11-01--How Iran Used Google To...,2019-11-01,zerohedge,How Iran Used Google To Disrupt 5% Of Global O...,Officials at Saudi Aramco believe that Iran us...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 21:45:00 +0000",1572659100,1572648627,2


# Keep only content, Reliability as columns

In [7]:
#Keep only content, Reliability as columns
totalData = totalData.drop(['id','date','source','title','author','url','published','published_utc','collection_utc'],axis=1)

In [8]:
totalData

,content,Reliability
0,"Last November, Stacey Abrams, a Democrat who r...",0
1,As lawmakers scramble to negotiate a border se...,0
2,Ten years after President Barack Obama signed ...,0
3,"President Donald Trump on Wednesday declared ""...",0
4,Republicans and Democrats on the exclusive com...,0
...,...,...
57152,It's unclear exactly when it happened (the BBG...,2
57153,Horse trading is the oxygen of politics; it is...,2
57154,Having plunged from over 10% to just 1% in the...,2
57155,Officials at Saudi Aramco believe that Iran us...,2


In [9]:
#find out unique Reliability labels type
totalData.Reliability.unique()

array([0, 2])

In [10]:
#Drop off mixed reliability(label=1) type
totalData = totalData[totalData.Reliability != 1]

In [11]:
#find out unique Reliability labels type
totalData.Reliability.unique()

array([0, 2])

In [33]:
def save_model(model_name, model_in):
    with open(model_name + ".pkl", 'wb') as file:
        pickle.dump(model_in, file)

# Cleaning dataset

In [12]:
totalData = totalData.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
totalData.content = totalData.content.apply(clean_text)
totalData.content = totalData.content.str.replace('\d+', '')

In [28]:
totalData.content.sample(10)

55762    struggling presidential candidate rep tim ryan...
32378    a cold winters day is the perfect opportunity ...
883      washington dc american civil rights groups hav...
29621    walk down any uk high street and you may do a ...
47439    a republican congressman in a rare move had so...
9491     maurizio sarris chelsea face the first of a se...
27486    i recently ordered makeup from revolution beau...
29186    debt collectors are almost universally recogni...
31598    sam allardyce has been criticised for his awfu...
30274    a website offering a student essaywriting serv...
Name: content, dtype: object

# Perform test train split

In [14]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
train_x, valid_x, train_y, valid_y = train_test_split(totalData['content'], totalData['Reliability'], test_size=0.20)

In [ ]:
save_model('valid_x', valid_x)
save_model('valid_y', valid_y)

# Develop Unigram and Bigram Tf-Idf feature vectors from data

In [15]:
# unigram level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=None)
tfidf_vect.fit(totalData['content'])
xtrain_tfidf_unigram =  tfidf_vect.transform(train_x)
xvalid_tfidf_unigram =  tfidf_vect.transform(valid_x)

# bigram level tf-idf (bigram in this case)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,2), max_features=None)
tfidf_vect_ngram.fit(totalData['content'])
xtrain_tfidf_bigram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_bigram =  tfidf_vect_ngram.transform(valid_x)


In [34]:
save_model("tfidf_vect", tfidf_vect)
save_model("tfidf_vect_ngram", tfidf_vect_ngram)

# Build Model functions

In [17]:
def train_model(classifier, feature_vector_train, label,  feature_vector_valid, valid_y,is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    #print("In Validation Data",metrics.accuracy_score(predictions, valid_y))
    print("f1 score: ",f1_score(valid_y,predictions,average='weighted'))        
    return metrics.accuracy_score(valid_y,predictions), classifier
    

# Base Line Model Used
## 1. Naive Bayes
## 2. Linear Classifier
## 3. Bagging
## 4. Boosting
## 5. SVM

#  Naive Bayes Model 

In [27]:
# Naive Bayes on Word Level TF IDF Vectors
print("For Unigram Tf-IDF features vectors using Naive Bayes")
accuracy, model= train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram, valid_y)
print ("Accuracy: ", accuracy)
save_model("Naive_bayes_word_level", model)

For Unigram Tf-IDF features vectors using Naive Bayes
f1 score:  0.6908281126919321
Accuracy:  0.7821028691392582


In [28]:
# Naive Bayes on Ngram Level TF IDF Vectors
print("For Bigram Tf-IDF features vectors using Naive Bayes")
accuracy, model = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram, valid_y)
print ("Accuracy: ", accuracy)
save_model("Naive_bayes_ngram", model)

For Bigram Tf-IDF features vectors using Naive Bayes
f1 score:  0.7025183236545448
Accuracy:  0.7872638208537439


# Linear Classifier

In [29]:
# Linear Classifier on Word Level TF IDF Vectors
print("For Unigram TF-Idf feature vectors using Logistic Regression")
accuracy,model = train_model(linear_model.LogisticRegression(max_iter=500), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram, valid_y)
print("accuracy: ", accuracy)
save_model("linear_classifier_word_level", model)

# Linear Classifier on Ngram Level TF IDF Vectors
print("For Biigram TF-Idf feature vectors using Logistic Regression")
accuracy,model = train_model(linear_model.LogisticRegression(max_iter=500),  xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram, valid_y)
print("Accuracy: ", accuracy)
save_model("linear_classifier_ngram", model)

For Unigram TF-Idf feature vectors using Logistic Regression
f1 score:  0.8733573332903743
accuracy:  0.8836599020293912
For Biigram TF-Idf feature vectors using Logistic Regression
f1 score:  0.8167309361588776
Accuracy:  0.8477081875437369


# Bagging Model

In [31]:
# RF on Word Level TF IDF Vectors
print("For Unigram Tf-Idf feature vectors using Random Forest Classifier")
accuracy,model = train_model(ensemble.RandomForestClassifier(n_jobs = -1), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram, valid_y)
print ("Accuracy: ", accuracy)
save_model("Bagging_word_level", model)
# RF on ngram Level TF IDF Vectors
print("For Bigram Tf-Idf feature vectors using Random Forest Classifier")
accuracy,model = train_model(ensemble.RandomForestClassifier(n_jobs = -1), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram, valid_y)
print ("Accuracy: ", accuracy)
save_model("Bagging_ngram", model)

For Unigram Tf-Idf feature vectors using Random Forest Classifier
f1 score:  0.7833724814724495
Accuracy:  0.8284639608117564
For Bigram Tf-Idf feature vectors using Random Forest Classifier
f1 score:  0.8101724162214172
Accuracy:  0.8444716585024493


# Boosting Model

In [35]:
print("For Unigram Tf-Idf feature vectors using Extreme Gradient Boosting")
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy,model = train_model(xgboost.XGBClassifier(), xtrain_tfidf_unigram.tocsc(), train_y, xvalid_tfidf_unigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)
save_model("Boosting_word_level", model)

# Extereme Gradient Boosting on ngram Level TF IDF Vectors
print("For Bigram Tf-Idf feature vectors using Extreme Gradient Boosting")
accuracy, model = train_model(xgboost.XGBClassifier(), xtrain_tfidf_bigram.tocsc(), train_y, xvalid_tfidf_bigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)
save_model("Boosting_ngram", model)

For Unigram Tf-Idf feature vectors using Extreme Gradient Boosting
f1 score:  0.8301802275473487
Accuracy:  0.854706088173548
For Bigram Tf-Idf feature vectors using Extreme Gradient Boosting
f1 score:  0.8313766026023136
Accuracy:  0.8573303009097271


# SVM Model

In [36]:
print("For Unigram Tf-Idf feature vectors using SVM")
#SVM Model on Unigram TF-IDF
accuracy, model = train_model(svm.SVC(), xtrain_tfidf_unigram.tocsc(), train_y, xvalid_tfidf_unigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)
save_model("SVM_word_level", model)

# SVM Model on Bigram TF-IDF
print("For Bigram Tf-Idf feature vectors using SVM")
accuracy, model = train_model(svm.SVC(), xtrain_tfidf_bigram.tocsc(), train_y, xvalid_tfidf_bigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)
save_model("SVM_ngram", model)

For Unigram Tf-Idf feature vectors using SVM


# Using Tf-idf features of Title and the Body of data and predictions using Multi Layer Perceptron

# Read Data again

In [2]:
data = pd.read_csv('../datasets/nela10.csv')

# Keep only title,body and reliability as columns 

In [3]:
data = data.drop(['id','date','source','author','url','published','published_utc','collection_utc'],axis=1)

# Now, how it looks like

In [4]:
data

,title,content,Reliability
0,"Who is Stacey Abrams, the Democrat who will re...","Last November, Stacey Abrams, a Democrat who r...",0
1,Ban government shutdowns? Some Republicans and...,As lawmakers scramble to negotiate a border se...,0
2,Democrats renew push for equal pay for equal work,Ten years after President Barack Obama signed ...,0
3,Trump attacks intel chiefs after they contradi...,"President Donald Trump on Wednesday declared ""...",0
4,Border security battle begins as Congress grap...,Republicans and Democrats on the exclusive com...,0
...,...,...,...
57152,Hong Kong Officer Faces Death Threats After Fi...,It's unclear exactly when it happened (the BBG...,2
57153,"Enough ""Quid Pro Quo"" Gaslighting!",Horse trading is the oxygen of politics; it is...,2
57154,"""Born For This? I Don't Think So"" - Trump Mock...",Having plunged from over 10% to just 1% in the...,2
57155,How Iran Used Google To Disrupt 5% Of Global O...,Officials at Saudi Aramco believe that Iran us...,2


# Drop off mixed reliability types

In [5]:
data = data[data.Reliability != 1]

In [6]:
#find out unique Reliability labels type
data.Reliability.unique()

array([0, 2])

# Clean data 

In [8]:
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data.content = data.content.apply(clean_text)
data.content = data.content.str.replace('\d+', '')


In [13]:
data['title']=data['title'].astype(str)

In [14]:
data.title = data.title.apply(clean_text)
data.title = data.title.str.replace('\d+', '')

In [16]:
#Cleaned data
data

,title,content,Reliability
0,who is stacey abrams the democrat who will res...,last november stacey abrams a democrat who ran...,0
1,ban government shutdowns some republicans and ...,as lawmakers scramble to negotiate a border se...,0
2,democrats renew push for equal pay for equal work,ten years after president barack obama signed ...,0
3,trump attacks intel chiefs after they contradi...,president donald trump on wednesday declared t...,0
4,border security battle begins as congress grap...,republicans and democrats on the exclusive com...,0
...,...,...,...
57152,hong kong officer faces death threats after fi...,its unclear exactly when it happened the bbg s...,2
57153,enough quid pro quo gaslighting,horse trading is the oxygen of politics it is ...,2
57154,born for this i dont think so trump mocks beto...,having plunged from over to just in the predic...,2
57155,how iran used google to disrupt of global oil...,officials at saudi aramco believe that iran us...,2


# Developing Tf-Idf features for title and content

In [19]:
# unigram level tf-idf
tfidf_vect1 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=None)
tfidf_vect1.fit(data['content'])
tfidf_vect2 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=None)
tfidf_vect2.fit(data['title'])
content_tfidf_unigram =  tfidf_vect1.transform(data['content'])
title_tfidf_unigram = tfidf_vect2.transform(data['title'])

# bigram level tf-idf (bigram in this case)
tfidf_vect_ngram1 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,2), max_features=None)
tfidf_vect_ngram1.fit(data['content'])
tfidf_vect_ngram2 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,2), max_features=None)
tfidf_vect_ngram2.fit(data['title'])
content_tfidf_bigram =  tfidf_vect_ngram1.transform(data['content'])
title_tfidf_bigram = tfidf_vect_ngram2.transform(data['title'])

In [22]:
#Check dimenstions
print(content_tfidf_unigram.shape)
print(title_tfidf_unigram.shape)
print(content_tfidf_bigram.shape)
print(title_tfidf_bigram.shape)

(57157, 737804)
(57157, 43040)
(57157, 6115724)
(57157, 311055)


In [25]:
fullData_unigram = np.empty((57157,780844))
fullData_bigram = np.empty((57157,6426779))

MemoryError: Unable to allocate 333. GiB for an array with shape (57157, 780844) and data type float64